<a href="https://colab.research.google.com/github/dbtjr1103/Basic_practice/blob/main/yolov8_74.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics

In [ ]:
!pip install roboflow

In [3]:
from IPython import display
display.clear_output()
!yolo mode=checks

Ultralytics YOLOv8.0.4 🚀 Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 23.0/166.8 GB disk)


In [4]:
from ultralytics import YOLO
from IPython.display import display, Image
import os
HOME = os.getcwd()
print(HOME)

/content


In [5]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 2651, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 2651 (delta 144), reused 218 (delta 97), pack-reused 2349
Receiving objects: 100% (2651/2651), 2.27 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (1708/1708), done.


In [6]:
cd ultralytics

/content/ultralytics


In [7]:
!pip install -qr requirements.txt

In [8]:
model = YOLO(f'{HOME}/yolov8n.pt')

  0%|          | 0.00/6.24M [00:00<?, ?B/s]

In [9]:
from roboflow import Roboflow
rf = Roboflow(api_key="qGwguWtA0jklwjG4hM6E")
project = rf.workspace("dairynew").project("dairy_new")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Dairy_New-1 in yolov8:: 100%|██████████| 2290/2290 [00:00<00:00, 7973.13it/s]


In [10]:
pwd

'/content/ultralytics'

In [14]:
!rm -rf /content/ultralytics/runs/detect

In [17]:
!yolo task=detect mode=train model=yolov8n.pt data=Dairy_New-1/data.yaml epochs=1 imgsz=160  --config-name=default.yaml --config-path=/content/ultralytics/ultralytics/yolo/configs

Ultralytics YOLOv8.0.4 🚀 Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=Dairy_New-1/data.yaml, epochs=1, patience=50, batch=16, imgsz=160, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=False, lr0=0.01, lrf=0.01, 

In [18]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/Dairy-Product-5/data.yaml epochs=100 imgsz=640  -config-name=default.yaml --config-path=/content/ultralytics/ultralytics/yolo/configs

usage: yolo
       [--help]
       [--hydra-help]
       [--version]
       [--cfg {job,hydra,all}]
       [--resolve]
       [--package PACKAGE]
       [--run]
       [--multirun]
       [--shell-completion]
       [--config-path CONFIG_PATH]
       [--config-name CONFIG_NAME]
       [--config-dir CONFIG_DIR]
       [--experimental-rerun EXPERIMENTAL_RERUN]
       [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
       [overrides [overrides ...]]
yolo: error: argument --cfg/-c: invalid choice: 'onfig-name=default.yaml' (choose from 'job', 'hydra', 'all')


In [ ]:
!yolo --help

cli is powered by Hydra.

== Configuration groups ==
Compose your configuration from those groups (group=option)



== Config ==
Override anything in the config (foo.bar=value)

task: detect
mode: train
model: null
data: null
epochs: 100
patience: 50
batch: 16
imgsz: 640
save: true
cache: false
device: null
workers: 8
project: null
name: null
exist_ok: false
pretrained: false
optimizer: SGD
verbose: false
seed: 0
deterministic: true
single_cls: false
image_weights: false
rect: false
cos_lr: false
close_mosaic: 10
resume: false
overlap_mask: true
mask_ratio: 4
dropout: 0.0
val: true
save_json: false
save_hybrid: false
conf: null
iou: 0.7
max_det: 300
half: false
dnn: false
plots: true
source: null
show: false
save_txt: false
save_conf: false
save_crop: false
hide_labels: false
hide_conf: false
vid_stride: 1
line_thickness: 3
visualize: false
augment: false
agnostic_nms: false
retina_masks: false
format: torchscript
keras: false
optimize: false
int8: false
dynamic: false
simplify: false


In [ ]:
from ultralytics.yolo.v8 import DetectionTrainer

class CustomTrainer(DetectionTrainer):
    def get_model(self, cfg, weights):
        ...

    def criterion(self, preds, batch):
        # get ground truth
        imgs = batch["imgs"]
        bboxes = batch["bboxes"]
        ...
        return loss, loss_items # see Reference-> Trainer for details on the expected format

# callback to upload model weights
def log_model(trainer):
    last_weight_path = trainer.last
    ...

trainer = CustomTrainer(overrides={...})
trainer.add_callback("on_train_epoch_end", log_model) # Adds to existing callback
trainer.train()